In [ ]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ultralytics

In [ ]:
with open("./IASA_Champ_Final/app_data/Almighty/1707228245/com.onmyway133.Almighty-setapp.json") as f:
    data = json.load(f)

image = cv2.imread("./IASA_Champ_Final/app_data/Almighty/1707228245/Almighty-1707228246.51.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
model = ultralytics.YOLO('best.pt')
cat_names = model.names

In [ ]:
def draw_elements(node, image):
    """
    :param node: JSON of GUI
    :param image_screenshot: cv2 image of GUI
    :return: cv2 image
    """
    if node['visible_bbox'] is not None:
        x, y, w, h = node['visible_bbox']
        yolo_res = model(cv2.resize(image[y:y+h,x:x+w,::-1], (244, 244)), verbose=False)
        top5 = [cat_names[i] for i in yolo_res[0].probs.top5]
        if np.sum(image[y:y+h, x:x+w] - image[y,x]) != 0 and node['role'] in top5:
            image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    if node['children']:
        for child in node['children']:
            image = draw_elements(child, image)
    
    return image

In [ ]:
new_image = image.copy()
new_image = draw_elements(data, new_image)

plt.imshow(new_image)
plt.axis('off')
plt.show()